In [1]:
# Import pandas 
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns


# Read in white wine data 
white = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv", sep=';')

# Read in red wine data 
red = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=';')

In [2]:
# Add `type` column to `red` with value 1
red['type'] = 1

# Add `type` column to `white` with value 0
white['type'] = 0

# Append `white` to `red`
wines = red.append(white, ignore_index=True)

In [3]:
y = wines.quality
X = wines.drop(['quality', 'residual sugar', 'free sulfur dioxide', 'type'], axis=1)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=46, stratify=y)

# Define the scaler 
scaler = StandardScaler().fit(X_train)

# Scale the train set
X_train_scaled = scaler.transform(X_train)

# Scale the test set
X_test_scaled = scaler.transform(X_test)

In [4]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras import layers, optimizers, regularizers
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras import metrics

from keras.utils import plot_model
#from kt_utils import *
import keras.backend as K
from sklearn import preprocessing, model_selection 

Using TensorFlow backend.


In [8]:
def create_model():
    # create model
    model = Sequential()
    # layer 1
    model.add(Dense(9, input_dim=9, activation='relu', kernel_initializer='normal') )
    #layer 2
    model.add(Dense(50, activation='relu', kernel_initializer='normal'))
    #layer 4
    model.add(Dense(1, activation='linear'))

    # Compile model
    model.compile(optimizer = 'adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasRegressor(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [100,150,200,250,300]
epochs = [5,10,20,40,60,80,100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_scaled, y_train)

In [9]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (abs(mean), stdev, param))

Best: -0.525015 using {'batch_size': 100, 'epochs': 100}
1.572110 (0.059532) with: {'batch_size': 100, 'epochs': 5}
0.859701 (0.062042) with: {'batch_size': 100, 'epochs': 10}
0.594925 (0.033192) with: {'batch_size': 100, 'epochs': 20}
0.544477 (0.023696) with: {'batch_size': 100, 'epochs': 40}
0.527681 (0.016331) with: {'batch_size': 100, 'epochs': 60}
0.528151 (0.012832) with: {'batch_size': 100, 'epochs': 80}
0.525015 (0.016092) with: {'batch_size': 100, 'epochs': 100}
2.312010 (0.153492) with: {'batch_size': 150, 'epochs': 5}
1.096744 (0.101546) with: {'batch_size': 150, 'epochs': 10}
0.756783 (0.062563) with: {'batch_size': 150, 'epochs': 20}
0.554815 (0.024501) with: {'batch_size': 150, 'epochs': 40}
0.533374 (0.016784) with: {'batch_size': 150, 'epochs': 60}
0.534271 (0.022964) with: {'batch_size': 150, 'epochs': 80}
0.530860 (0.015998) with: {'batch_size': 150, 'epochs': 100}
7.035172 (0.957605) with: {'batch_size': 200, 'epochs': 5}
1.536351 (0.128566) with: {'batch_size': 200

In [10]:
#Use scikit-learn to grid search the number of neurons
def create_model(neurons=1):
    # create model
    model = Sequential()
    # layer 1
    model.add(Dense(9, input_dim=9, activation='relu', kernel_initializer='normal'))
    #layer 2
    model.add(Dense(neurons, activation='relu', kernel_initializer='normal'))
    model.add(Dropout(0.5))
    #layer 3
    model.add(Dense(1, activation='linear'))
    
    # Compile model
    model.compile(optimizer = 'adam', loss='mean_squared_error', metrics=['accuracy'])
    return model


# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasRegressor(build_fn=create_model, epochs=100, batch_size=100, verbose=0)
# define the grid search parameters
neurons = [1, 5, 10, 20, 40, 60, 80, 100]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_scaled, y_train)

In [11]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.552366 using {'neurons': 100}
-3.871111 (2.040323) with: {'neurons': 1}
-1.300060 (0.158649) with: {'neurons': 5}
-0.948730 (0.098481) with: {'neurons': 10}
-0.657123 (0.028053) with: {'neurons': 20}
-0.594570 (0.004370) with: {'neurons': 40}
-0.562028 (0.019666) with: {'neurons': 60}
-0.553586 (0.022676) with: {'neurons': 80}
-0.552366 (0.011894) with: {'neurons': 100}


In [12]:
def create_model(hidden_layers=1):
    # create model
    model = Sequential()
    # layer 1
    model.add(Dense(9, input_dim=9, activation='relu',kernel_initializer='normal'))
    
    #hidden layers
    for i in range(hidden_layers):
        # Add one hidden layer
        model.add(Dense(100, activation='relu',  kernel_initializer='normal'))
    
    #layer 4
    model.add(Dense(1, activation='linear'))
    
    # Compile model
    model.compile(optimizer = 'adam', loss='mean_squared_error', metrics=['accuracy'])
    return model


# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasRegressor(build_fn=create_model, batch_size=100, verbose=0, epochs=100)
# define the grid search parameters
hidden_layers = [1,2,3,4]
param_grid = dict(hidden_layers = hidden_layers)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_scaled, y_train)


In [13]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.513021 using {'hidden_layers': 4}
-0.523631 (0.017416) with: {'hidden_layers': 1}
-0.520083 (0.018230) with: {'hidden_layers': 2}
-0.535171 (0.025213) with: {'hidden_layers': 3}
-0.513021 (0.022293) with: {'hidden_layers': 4}


In [16]:
# Use scikit-learn to grid search the activation function
def create_model(activation='relu'):
    # create model
    model = Sequential()
    # layer 1
    model.add(Dense(9, input_dim=9, activation='relu', kernel_initializer='normal'))
    #layer 2
    model.add(Dense(100, activation=activation,kernel_initializer='normal'))
    #layer 3
    model.add(Dense(100, activation=activation,kernel_initializer='normal'))
    #layer 4
    model.add(Dense(1, activation='linear'))
    
    # Compile model
    model.compile(optimizer = 'adam', loss='mean_squared_error', metrics=['accuracy'])
    return model


# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasRegressor(build_fn=create_model, epochs=100, batch_size=100, verbose=0)
# define the grid search parameters
activation = ['relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_scaled, y_train)

In [17]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.529537 using {'activation': 'relu'}
-0.529537 (0.024471) with: {'activation': 'relu'}
-0.530304 (0.025810) with: {'activation': 'tanh'}
-0.536448 (0.025079) with: {'activation': 'sigmoid'}
-0.532614 (0.021519) with: {'activation': 'hard_sigmoid'}
-0.539701 (0.026364) with: {'activation': 'linear'}


In [19]:
from keras.optimizers import Adam
def create_model(learn_rate=0.01, momentum=0):
    # create model
    model = Sequential()
    # layer 1
    model.add(Dense(9, input_dim=9, activation='relu', kernel_initializer='normal'))
    #layer 2
    model.add(Dense(100, activation='relu', kernel_initializer='normal'))
    #layer 3
    model.add(Dense(100, activation='relu',kernel_initializer='normal'))
    #layer 4
    model.add(Dense(1, activation='linear'))
    optimizer = Adam(lr=learn_rate)
    # Compile model
    model.compile(optimizer = optimizer, loss='mean_squared_error', metrics=['accuracy'])
    return model

import numpy as np
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasRegressor(build_fn=create_model, epochs=100, batch_size=100, verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(learn_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_scaled, y_train)

In [20]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.531070 using {'learn_rate': 0.001}
-0.531070 (0.027230) with: {'learn_rate': 0.001}
-0.532817 (0.015435) with: {'learn_rate': 0.01}
-0.552399 (0.035265) with: {'learn_rate': 0.1}
-0.770510 (0.032409) with: {'learn_rate': 0.2}
-0.773951 (0.032957) with: {'learn_rate': 0.3}
